### Data Classes and Manual Validation

In [ ]:
from dataclasses import dataclass, field
from typing import List

@dataclass
class User:
    name: str
    age: int
    country: str = "India"
    tags: List[str] = field(default_factory=list)

    def __post_init__(self):
        if self.age < 0:
            raise ValueError("Age cannot be negative")

    def is_adult(self):
        return self.age >= 18

@dataclass(frozen=True)
class ImmutableUser:
    name: str
    age: int

@dataclass(slots=True)
class SlotUser:
    name: str
    age: int

user1 = User("Alice", 25)
user2 = User("Bob", 17, "USA")
user3 = User("Charlie", -5)

print(user1.is_adult())
print(user2.is_adult())

immutable_user = ImmutableUser("Dave", 30)

slot_user = SlotUser("Eve", 28)
print(slot_user.name)

user4 = User("Alice", 25)
print(user1 == user4)

###  Validation with pydantic

In [ ]:
from pydantic import BaseModel, validator, conint, constr, ValidationError
from typing import Optional

class Profile(BaseModel):
    bio: str
    website: Optional[str] = None

class User(BaseModel):
    name: str
    age: int
    username: constr(min_length=3)
    profile: Optional[Profile] = None

    @validator('name')
    def name_must_be_capitalized(cls, v):
        if not v[0].isupper():
            raise ValueError('Name must be capitalized')
        return v

try:
    user = User(**{"name": "Bob", "age": "not a number", "username": "bob"})
except ValidationError as e:
    print(e)

user_data = {
    "name": "Alice",
    "age": "25",
    "username": "alice123",
    "profile": {
        "bio": "Software developer",
        "website": "example.com"
    }
}

user = User(**user_data)
print(user.dict())
print(user.json())

class StrictUser(BaseModel):
    name: str
    age: conint(gt=0)
    username: constr(min_length=3)

strict_user = StrictUser(name="Charlie", age=30, username="charlie")

### Field Metadata and Readability

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional

class User(BaseModel):

    id: int = Field(..., alias="user_id", description="Unique identifier for the user")
    email: str = Field(..., description="User's email address", example="user@example.com")
    name: str = Field(..., title="Full Name", description="User's full name", example="John Doe")
    age: Optional[int] = Field(None, ge=0, description="User's age in years", example=30)
    is_active: bool = Field(True, description="Whether the user account is active")

user_data = {
    "user_id": 123,
    "email": "john@example.com",
    "name": "John Smith",
    "age": 35
}

user = User(**user_data)
print(user.json(by_alias=True))

### Logging Best Practices

In [ ]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('app.log'),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(__name__)

def process_user(user):
    logger.debug(f"Processing user: {user}")
    logger.info(f"User created: {user['name']}")
    if user['age'] < 0:
        logger.warning(f"Invalid age for user: {user['name']}")
    try:
        result = 100 / user.get('level', 0)
    except Exception as e:
        logger.error(f"Calculation failed: {e}")

user_data = {'name': 'Alice', 'age': 25, 'level': 2}
process_user(user_data)

user_data_bad = {'name': 'Bob', 'age': -5}
process_user(user_data_bad)

logger.info("Application finished")

###  Code Clarity and Naming

In [ ]:
"""Module containing functions for processing user data and calculating scores."""

MAX_RETRIES = 3
TIMEOUT_SECONDS = 30

def calculate_user_score(user_data):
    if not is_valid_user(user_data):
        return 0
    return sum(get_metric_values(user_data))

def is_valid_user(user_data):
    return has_required_fields(user_data) and is_active_user(user_data)

def has_required_fields(user_data):
    required = ['name', 'email', 'age']
    return all(field in user_data for field in required)

def is_active_user(user_data):
    return user_data.get('status') == 'active'

def get_metric_values(user_data):
    metrics = []
    if can_access_service(user_data):
        metrics.append(5)
    if has_premium_access(user_data):
        metrics.append(10)
    return metrics

def can_access_service(user_data):
    return user_data['age'] >= 18

def has_premium_access(user_data):
    return user_data.get('membership') == 'premium'

def process_with_retries(data):
    for attempt in range(MAX_RETRIES):
        try:
            return send_data_to_service(data)
        except ConnectionError:
            if attempt == MAX_RETRIES - 1:
                raise

def send_data_to_service(data):
    import time
    time.sleep(TIMEOUT_SECONDS)
    return True